<a href="https://colab.research.google.com/github/GouthamiKatha/DBS_table9/blob/main/MINI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#new final
!pip install PyPDF2[crypto]
!pip install 'PyPDF2<3.0'
!pip install pypdf2
!pip install docx2txt
from PyPDF2 import PdfReader
import docx2txt
import re
import numpy as np
import pandas as pd
import io
from PyPDF2 import PdfFileReader, PdfFileWriter, PdfFileMerger
import spacy
import en_core_web_sm
from spacy.matcher import Matcher
import nltk
import spacy
nltk.download('stopwords')
from nltk.corpus import stopwords
from google.colab import files
# load pre-trained model
nlp = en_core_web_sm.load()

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)
# Grad all general stop words
STOPWORDS = set(stopwords.words('english'))

# Education Degrees
EDUCATION = [
            'BE','B.E.', 'B.E', 'BS', 'B.S',
            'ME', 'M.E', 'M.E.', 'M.B.A', 'MBA', 'MS', 'M.S',
            'BTECH', 'B-TECH', 'M.TECH', 'MTECH',
            'SSLC', 'SSC' 'HSC', 'CBSE', 'ICSE', 'X', 'XII','B.TECH','INTERMEDIATE','SSC'
        ]
#Extracting text from DOCX
def doctotext(m):
    temp = docx2txt.process(m)
    resume_text = [line.replace('\t', ' ') for line in temp.split('\n') if line]
    text = ' '.join(resume_text)
    return (text)

#Extracting text from PDF
def pdftotext(m):
    # pdf file object
    # you can find find the pdf file with complete code in below
    pdfFileObj = open(m,'rb')

    # pdf reader object
    pdfFileReader = PdfReader(pdfFileObj)

    # number of pages in pdf
    num_pages = pdfFileReader.numPages

    currentPageNumber = 0
    text = ''

    # Loop in all the pdf pages.
    while(currentPageNumber < num_pages ):

        # Get the specified pdf page object.
        pdfPage = pdfFileReader.getPage(currentPageNumber)

        # Get pdf page text.
        text = text + pdfPage.extractText()

        # Process next page.
        currentPageNumber += 1
    return (text)

def extract_name(resume_text):
    nlp_text = nlp(resume_text)

    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]

    matcher.add('NAME',[pattern],on_match=None)

    matches = matcher(nlp_text)

    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text

def extract_mobile_number(resume_text):
    phone = re.findall(re.compile(r"(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?"), resume_text)

    if phone:
        number = ''.join(phone[0])
        if len(number) > 10:
            return number
        else:
            return number
def extract_email_addresses(string):
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    return r.findall(string)

def extract_education(resume_text):
    nlp_text = nlp(resume_text)

    # Sentence Tokenizer
    #nlp_text = [sent.string.strip() for sent in nlp_text.sents]
    nlp_text = [sent.text.strip() for sent in nlp_text.sents]

    edu = {}
    # Extract education degree
    for index, text in enumerate(nlp_text):
        for tex in text.split():
            # Replace all special symbols
            tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            if tex.upper() in EDUCATION and tex not in STOPWORDS:
                edu[tex] = text + nlp_text[index ]



    # Extract year
    education = []
    for key in edu.keys():

        year = re.search(re.compile(r'(((20|19)(\d{})))'), edu[key])
        if year:
            education.append((key, ''.join(year[0])))
        else:
            education.append(key)
    return education

#Skills
nlp = spacy.load('en_core_web_sm')
#NOUN_CHUNKS = nlp.noun_chunks
def extract_skills(resume_text):
    #nlp = spacy.load('en_core_web_md')
    #doc = nlp(text)

    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    colnames = ['Skills']
    # reading the csv file
    #up=files.upload()
    data = pd.read_csv('skills.csv', names=colnames)
    #up=files.upload()
    #data=pd.read_csv(io.BytesIO(up['skills.csv']))
    # extract values
    skill = data.Skills.tolist()
    print(skill)
    skillset = []

    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in skill:
          skillset.append(token)


    for token in nlp_text.noun_chunks:
        token = token.text.lower().strip()
        if token in skill:
            skillset.append(token)
    return [i.capitalize() for i in set([i.lower() for i in skillset])]


#main function
if __name__ == '__main__':

    up=files.upload()
    FilePath = "Gouthami's Resume.pdf"
    FilePath.lower().endswith(('.png', '.docx'))
    if FilePath.endswith('.docx'):
      textinput = doctotext(FilePath)
    elif FilePath.endswith('.pdf'):
      textinput = pdftotext(FilePath)
    else:
      print("File not support")
    #print(textinput)
    print('Name: ',extract_name(textinput))
    print('Mobile Number:',extract_mobile_number(textinput))
    print('Mail id: ',extract_email_addresses(textinput))
    print('Qualification: ',extract_education(textinput))
    print ('Skills:',extract_skills(textinput))
    #print(textinput)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Saving Gouthami's Resume.pdf to Gouthami's Resume (7).pdf
Name:  Gouthami Katha
Mobile Number: None
Mail id:  ['kathagouthami16@gmail.com']
Qualification:  ['B-Tech', 'Intermediate', 'SSC']
['Skills', 'C', 'Java', 'Python', 'React', 'C++', 'Sql']
Skills: []


In [ ]:
from google.colab import files
import pandas as pd
import spacy
import numpy as np
import io

#nlp = spacy.load('en_core_web_sm')
#noun_chunks = nlp.noun_chunks
nlp = spacy.load('en_core_web_sm')
#NOUN_CHUNKS = nlp.noun_chunks


def extract_skills(resume_text):
    #nlp = spacy.load('en_core_web_md')
    #doc = nlp(text)

    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    colnames = ['skill']
    # reading the csv file
    data = pd.read_csv('skills.csv', names=colnames)
    up=files.upload()
    #data=pd.read_csv(io.BytesIO(up['skills.csv']))
    # extract values
    skills = data.skill.tolist()
    print(skills)
    skillset = []

    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)

    for token in nlp.noun_chunks:
        token = token.text.lower().strip()
        if token in skills:
            skillset.append(token)
    return [i.capitalize() for i in set([i.lower() for i in skillset])]

print ('Skills',extract_skills("Java"))

FileNotFoundError: ignored

In [ ]:
#final code
!pip install pypdf2
!pip install docx2txt
import docx2txt
import re
import numpy as np
import pandas as pd
import io
from PyPDF2 import PdfFileReader, PdfFileWriter, PdfFileMerger
import spacy
import en_core_web_sm
from spacy.matcher import Matcher
import nltk
import spacy
nltk.download('stopwords')
from nltk.corpus import stopwords
from google.colab import files
# load pre-trained model
nlp = en_core_web_sm.load()

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)
# Grad all general stop words
STOPWORDS = set(stopwords.words('english'))

# Education Degrees
EDUCATION = [
            'BE','B.E.', 'B.E', 'BS', 'B.S',
            'ME', 'M.E', 'M.E.', 'M.B.A', 'MBA', 'MS', 'M.S',
            'BTECH', 'B-TECH', 'M.TECH', 'MTECH',
            'SSLC', 'SSC' 'HSC', 'CBSE', 'ICSE', 'X', 'XII','B.TECH','INTERMEDIATE','SSC'
        ]
#Extracting text from DOCX
def doctotext(m):
    temp = docx2txt.process(m)
    resume_text = [line.replace('\t', ' ') for line in temp.split('\n') if line]
    text = ' '.join(resume_text)
    return (text)

#Extracting text from PDF
def pdftotext(m):
    # pdf file object
    # you can find find the pdf file with complete code in below
    pdfFileObj = open(m,'rb')

    # pdf reader object
    pdfFileReader = PdfFileReader(pdfFileObj)

    # number of pages in pdf
    num_pages = pdfFileReader.numPages

    currentPageNumber = 0
    text = ''

    # Loop in all the pdf pages.
    while(currentPageNumber < num_pages ):

        # Get the specified pdf page object.
        pdfPage = pdfFileReader.getPage(currentPageNumber)

        # Get pdf page text.
        text = text + pdfPage.extractText()

        # Process next page.
        currentPageNumber += 1
    return (text)

def extract_name(resume_text):
    nlp_text = nlp(resume_text)

    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]

    matcher.add('NAME',[pattern],on_match=None)

    matches = matcher(nlp_text)

    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text

def extract_mobile_number(resume_text):
    phone = re.findall(re.compile(r"(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?"), resume_text)

    if phone:
        number = ''.join(phone[0])
        if len(number) > 10:
            return number
        else:
            return number
def extract_email_addresses(string):
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    return r.findall(string)

def extract_education(resume_text):
    nlp_text = nlp(resume_text)

    # Sentence Tokenizer
    #nlp_text = [sent.string.strip() for sent in nlp_text.sents]
    nlp_text = [sent.text.strip() for sent in nlp_text.sents]

    edu = {}
    # Extract education degree
    for index, text in enumerate(nlp_text):
        for tex in text.split():
            # Replace all special symbols
            tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            if tex.upper() in EDUCATION and tex not in STOPWORDS:
                edu[tex] = text + nlp_text[index ]



    # Extract year
    education = []
    for key in edu.keys():

        year = re.search(re.compile(r'(((20|19)(\d{})))'), edu[key])
        if year:
            education.append((key, ''.join(year[0])))
        else:
            education.append(key)
    return education

#Skills
nlp = spacy.load('en_core_web_sm')
#NOUN_CHUNKS = nlp.noun_chunks
def extract_skills(resume_text):
    #nlp = spacy.load('en_core_web_md')
    #doc = nlp(text)

    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    colnames = ['skill']
    # reading the csv file
    data = pd.read_csv('skill.csv', names=colnames)
    #up=files.upload()
    #data=pd.read_csv(io.BytesIO(up['skill.csv']))
    # extract values
    skills = data.skill.tolist()
    print(skills)
    skillset = []

    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in skills:
          skillset.append(token)


    for token in nlp_text.noun_chunks:
        token = token.text.lower().strip()
        if token in skills:
            skillset.append(token)
    return [i.capitalize() for i in set([i.lower() for i in skillset])]


#main function
if __name__ == '__main__':

    #up=files.upload()
    FilePath = 'Gouthami Resume.docx'
    FilePath.lower().endswith(('.png', '.docx'))
    if FilePath.endswith('.docx'):
      textinput = doctotext(FilePath)
    elif FilePath.endswith('.pdf'):
      textinput = pdftotext(FilePath)
    else:
      print("File not support")
    #print(textinput)
    print('Name: ',extract_name(textinput))
    print('Mobile Number: ',extract_mobile_number(textinput))
    print('Mail id: ',extract_email_addresses(textinput))
    print('Qualification: ',extract_education(textinput))
    print ('Skills',extract_skills(textinput))
    #print(textinput)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Name:  University Year
Mobile Number:  939077684
Mail id:  ['kathagouthami16@gmail']
Qualification:  ['BTech', 'Intermediate', 'SSC']
['Skills', 'java', 'c', 'c++', 'dbms', 'aws', 'frontend', 'python']
Skills ['Python', 'C++', 'Java', 'C']


In [ ]:
#final code

!pip install pypdf2
!pip install docx2txt
import docx2txt
import re
import numpy as np
import pandas as pd
import io
from PyPDF2 import PdfFileReader, PdfFileWriter, PdfFileMerger
import spacy
import en_core_web_sm
from spacy.matcher import Matcher
import nltk
import spacy
nltk.download('stopwords')
from nltk.corpus import stopwords
from google.colab import files
# load pre-trained model
nlp = en_core_web_sm.load()

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)
# Grad all general stop words
STOPWORDS = set(stopwords.words('english'))

# Education Degrees
EDUCATION = [
            'BE','B.E.', 'B.E', 'BS', 'B.S',
            'ME', 'M.E', 'M.E.', 'M.B.A', 'MBA', 'MS', 'M.S',
            'BTECH', 'B-TECH', 'M.TECH', 'MTECH',
            'SSLC', 'SSC' 'HSC', 'CBSE', 'ICSE', 'X', 'XII','B.TECH','INTERMEDIATE','SSC'
        ]
#Extracting text from DOCX
def doctotext(m):
    temp = docx2txt.process(m)
    resume_text = [line.replace('\t', ' ') for line in temp.split('\n') if line]
    text = ' '.join(resume_text)
    return (text)

#Extracting text from PDF
def pdftotext(m):
    # pdf file object
    # you can find find the pdf file with complete code in below
    pdfFileObj = open(m,'rb')

    # pdf reader object
    pdfFileReader = PdfFileReader(pdfFileObj)

    # number of pages in pdf
    num_pages = pdfFileReader.numPages

    currentPageNumber = 0
    text = ''

    # Loop in all the pdf pages.
    while(currentPageNumber < num_pages ):

        # Get the specified pdf page object.
        pdfPage = pdfFileReader.getPage(currentPageNumber)

        # Get pdf page text.
        text = text + pdfPage.extractText()

        # Process next page.
        currentPageNumber += 1
    return (text)

def extract_name(resume_text):
    nlp_text = nlp(resume_text)

    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]

    matcher.add('NAME',[pattern],on_match=None)

    matches = matcher(nlp_text)

    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text

def extract_mobile_number(resume_text):
    phone = re.findall(re.compile(r"(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?"), resume_text)

    if phone:
        number = ''.join(phone[0])
        if len(number) > 10:
            return number
        else:
            return number
def extract_email_addresses(string):
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    return r.findall(string)

def extract_education(resume_text):
    nlp_text = nlp(resume_text)

    # Sentence Tokenizer
    #nlp_text = [sent.string.strip() for sent in nlp_text.sents]
    nlp_text = [sent.text.strip() for sent in nlp_text.sents]

    edu = {}
    # Extract education degree
    for index, text in enumerate(nlp_text):
        for tex in text.split():
            # Replace all special symbols
            tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            if tex.upper() in EDUCATION and tex not in STOPWORDS:
                edu[tex] = text + nlp_text[index ]



    # Extract year
    education = []
    for key in edu.keys():

        year = re.search(re.compile(r'(((20|19)(\d{})))'), edu[key])
        if year:
            education.append((key, ''.join(year[0])))
        else:
            education.append(key)
    return education

#Skills
nlp = spacy.load('en_core_web_sm')
#NOUN_CHUNKS = nlp.noun_chunks
def extract_skills(resume_text):
    #nlp = spacy.load('en_core_web_md')
    #doc = nlp(text)

    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    colnames = ['skill']
    # reading the csv file
    data = pd.read_csv('skill.csv', names=colnames)
    #up=files.upload()
    #data=pd.read_csv(io.BytesIO(up['skill.csv']))
    # extract values
    skills = data.skill.tolist()
    #print(skills)
    skillset = []

    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in skills:
          skillset.append(token)


    for token in nlp_text.noun_chunks:
        token = token.text.lower().strip()
        if token in skills:
            skillset.append(token)
    return [i.capitalize() for i in set([i.lower() for i in skillset])]


#main function
if __name__ == '__main__':

    FilePath=files.upload()
    FilePath = 'GouthamiResume.pdf'
    FilePath.lower().endswith(('.png', '.docx'))
    if FilePath.endswith('.docx'):
      textinput = doctotext(FilePath)
    elif FilePath.endswith('.pdf'):
      textinput = pdftotext(FilePath)
    else:
      print("File not support")
    #print(textinput)
    print('Name: ',extract_name(textinput))
    print('Mobile Number: ',extract_mobile_number(textinput))
    print('Mail id: ',extract_email_addresses(textinput))
    print('Qualification: ',extract_education(textinput))
    print ('Skills:',extract_skills(textinput))
    #print(textinput)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Saving GouthamiResume.pdf to GouthamiResume (5).pdf
Name:  Gouthami Katha
Mobile Number:  None
Mail id:  ['kathagouthami16@gmail.com']
Qualification:  ['B-Tech', 'Intermediate', 'SSC']


FileNotFoundError: ignored